## These cells are used to help clean up and generate training data for a self-steering vehicle in Unreal 4.

## Description

The cells below are intended to be run individually based on the how the training data was generated. I experimented with various versions and approaches, explained above each cell.

I'm using the fast.ai MOOC Practical Deep Learning for Coders, version 3 as a basis for these experiments. The third lesson deals with image segmentation and regression. To make my life easier and so I can reuse their code, I'm formatting my image segmentation data similarly the dataset they use for the course. That dataset can be found at: http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import os

## A.1: Read in steering values from Unreal log file as a dataframe. Clean it up and write to a csv

steeringValues.txt is derived from the log file that Unreal generates at runtime. Each frame, a script in Unreal outputs the steering values (ranges from -1 for left steer, to +1 for right steer) from the game controller to the log file. The log file contains lots of other data and output, though, so I copy and paste only the lines I need into the txt file, and use the lines of code below to cleanup and format the data for a csv file that Unreal can read.

I eventually abandoned this approach because the data I collected was often undesireable behavior. For example, from the center of the road, if I drove to the edge and then swerved back in toward center, I wouldn't want to collect the initial steering values or screenshots because this is not the kind of behavior that I'd like to teach my steerer. I don't want my vehicle to wildly swing to one side of the road when it's driving nice and smoothly down the center. However, I do need data that tells the vehicle what to do when it does find itself near the edge of the road and wants to steer back toward center. I couldn't figure out a smart way to exclude the unwanted data collection though, so I moved on to a different approach: A.2.

In [2]:
path = Path('.')
df = pd.read_csv(path/'steeringValues.txt', sep=']')
df = df.drop(df.columns[[0, 1, 2]], axis=1)
df.columns = ['Axis Value'] #rename column
df.to_csv(path/'steeringValues.csv') #output to csv file
df.shape

(3260, 1)

## A.2: Create steering values csv from ranges instead of from Unreal output

Using a simple straight road with lane markings on each side, I drove the vehicle down the center of the road for 100 frames, capturing screenshots every frame. For the steering values, I generated small random values between -0.1 and 0.1. This training data represents the behavior I want the vehicle to have when it's in the center and looking directly down the road, thus the small steering values. I then oriented the vehicle almost perpendicular to the lane markings, and began collecting 80 frames as I gradually steered back to the center of the road. I then generated 80 evenly spaced values between -1 and 0 for the steering values. This represents the behavior I want the vehicle to perform when it finds itself near the center of the road, but not looking directly down the road. I then repeated the same thing for the left side of the road, but used steering values arranged evenly from 1 to 0. I then needed data for when the vehicle found itself on either side of the road, looking directly down the road. I captured frames of the vehicle placed near the edge of the road and steering gradually back to center, and gave them a steering value arranged evenly from 0.5 to 0 for the left side, and -0.5 to 0 for the right.

In [ ]:
path = Path('./data/imageSeg')
df = pd.DataFrame(np.random.uniform(-0.1,0.1,(655)), columns = ['Axis Value'], dtype=float)
df.iloc[100:180]['Axis Value'] = np.arange(-1.0, 0, 0.0125) #generate evenly spaced values between -1 and 0
df.iloc[180:259]['Axis Value'] = np.flip(np.arange(0.0, 1.0, 0.0127))
df.iloc[259:484]['Axis Value'] = np.flip(np.arange(0.0, 0.5, 0.00223))
df.iloc[484:655]['Axis Value'] = np.arange(-0.5, 0.0, 0.00293)
df.to_csv(path/'steeringValues.csv') #output to csv file
df.shape

## B.1: Read in the mask frame output from Unreal. Convert to a segmented image with coded masks.

Image segmentation. I got really good and accurate results, but it was too slow making predictions. I didn't use any segmentation in the end result because I needed to update the steering values several times a second to stay on the road, and the predictions were taking over a second. I want to revisit this later and see if I can get the faster predictor someday.

Method: capture screenshots from Unreal that have flat-shaded materials assigned to objects that I'd like to mask. Convert screenshot to an array, extract color values into individual variables, make an array mask for color ranges that are near the color values for each object, then change the color values to the masked code value. Will only work as long as the original color values are not too close to the coded values, so make sure that when generating the segmented image that you don't pick grays that are too close to black. Pick bright vibrant colors.

Commented out cells were previous attempts at generating the mask. They were too slow, taking many hours to process the frames. The makeMaskFast function below was really quick and I got fairly clean results.

In [5]:
#old make mask function. Much faster version makeMaskFast below.
# def makeMask(imgArray):
    
#     # Mask objects and their codes
#     sky = [163, 163, 163, 255]              #Code=1
#     tree = [233, 168, 0, 255]               #Code=2
#     road = [0, 12, 221, 255]               #Code=3
#     fence = [225, 228, 11, 255]             #Code=4
#     objectStationary = [238, 57, 223, 255]  #Code=5
#     grass = [0, 232, 0, 255]               #Code=6
#     lightpost = [92, 222, 223, 255]         #Code=7
#     house = [236, 0, 7, 255]               #Code=8
#     arrow = [234, 120, 177, 255]            #Code=9

#     mask = np.zeros((361, 760, 3), dtype=np.uint8) #initialize mask to zeros
    
#      #for each pixel, check if it's close to the color of the mask object (values above). If it is, then make mask pixel
#      #equal to mask object code number. Return the mask.
    
#     for row in range(0, imgArray.shape[0]):        
#         for col in range(0, imgArray.shape[1]):
            
#             if (np.allclose(imgArray[row][col], sky, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([1, 1, 1])
#                 continue
#             if (np.allclose(imgArray[row][col], tree, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([2, 2, 2])
#                 continue
#             if (np.allclose(imgArray[row][col], road, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([3, 3, 3])
#                 continue
#             if (np.allclose(imgArray[row][col], fence, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([4, 4, 4])
#                 continue
#             if (np.allclose(imgArray[row][col], objectStationary, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([5, 5, 5])
#                 continue
#             if (np.allclose(imgArray[row][col], grass, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([6, 6, 6])
#                 continue
#             if (np.allclose(imgArray[row][col], lightpost, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([7, 7, 7])
#                 continue
#             if (np.allclose(imgArray[row][col], house, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([8, 8, 8])
#                 continue
#             if (np.allclose(imgArray[row][col], arrow, rtol=.25, atol=10)):
#                 mask[row][col] = np.uint8([9, 9, 9])          

#     return mask

In [8]:
# first attempt at mask generation. Was really slow because it read each pixel for each mask object.

# def makeMask(imgArray, maskObject, maskObjectCode):
#     #for each pixel, check if it's close to the color of the maskObject. If it is, then make pixel = (1, 1, 1), if not make 
#     #pixel = (0, 0, 0). Return the product of the pixel array and the code of the maskObject to create the coded mask.

#     mask = np.zeros((361, 760, 3), dtype=np.uint8)

#     for row in range(0, imgArray.shape[0]):
#         for col in range(0, imgArray.shape[1]):
#             mask[row][col] = int(np.allclose(imgArray[row][col], maskObject, rtol=.25, atol=10))
    
#     return np.uint8(mask*maskObjectCode)

In [255]:

# image = Image.open('./labels/screenshots/ScreenShot00010.png').resize((760, 361)).convert(mode='RGB')
# #Code dictionary - name:(r, g, b, code)
# codeDict = {'sky':(163, 163, 163, 1),
#             'tree':(233, 168, 0, 2),
#             'road':(0, 12, 221, 3),
#             'fence':(225, 228, 11, 4),
#             'objectStationary':(238, 57, 223, 5),
#             'grass':(0, 232, 0, 6),
#             'lighpost':(92, 222, 223, 7),
#             'house':(236, 0, 7, 8),
#             'arrow':(234, 120, 177, 9)}

# data = np.asarray(image)
# data.flags.writeable = True #Don't know why the array isn't writeable. Had to explicity set it.

# red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]

# #For each mask object replace colors with coded mask.
# for key, value in codeDict.items():

#     r1, g1, b1, code = value

#     mask = (np.isclose(red, r1, rtol=0.25, atol=15)) & (np.isclose(green, g1, rtol=0.25, atol=15)) & \
#         (np.isclose(blue, b1, rtol=0.25, atol=15))

#     data[:,:,:3][mask] = [code, code, code]

# mask = (data > 9) | (data[:,:,:1] != data[:,:,:3]) #Create a mask for remaining 'void' colors.
# #Make another mask for remaining red that is < 9 and not uniform(there's got to be a better way)
# mask1 = (data[:,:,0] != data[:,:,1]) | (data[:,:,0] != data[:,:,2]) | (data[:,:,1] != data[:,:,2])
# mask[:,:,0] = mask1[:,:]
# mask[:,:,1] = mask1[:,:]
# mask[:,:,2] = mask1[:,:]
# data[:,:,:3][mask] = 0 #Set 'void' colors to black.
# img = Image.fromarray(data, mode='RGB')
# img.save('./temp.png', 'png')


array([[[False, False, False],
        [ True,  True,  True],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [ True,  True,  True],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [ True,  True,  True],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       ...,

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False,

In [256]:
def makeMaskFast(image):

    #Code dictionary - name:(r, g, b, code)
    codeDict = {'sky':(163, 163, 163, 1),
                'tree':(233, 168, 0, 2),
                'road':(0, 12, 221, 3),
                'fence':(225, 228, 11, 4),
                'objectStationary':(238, 57, 223, 5),
                'grass':(0, 232, 0, 6),
                'lighpost':(92, 222, 223, 7),
                'house':(236, 0, 7, 8),
                'arrow':(234, 120, 177, 9)}

    data = np.asarray(image)
    data.flags.writeable = True #Don't know why the array isn't writeable. Had to explicity set it.

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]

    #For each mask object replace colors with coded mask.
    for key, value in codeDict.items():

        r1, g1, b1, code = value

        mask = (np.isclose(red, r1, rtol=0.25, atol=15)) & (np.isclose(green, g1, rtol=0.25, atol=15)) & \
            (np.isclose(blue, b1, rtol=0.25, atol=15))

        data[:,:,:3][mask] = [code, code, code]

    mask = (data > 9) | (data[:,:,:1] != data[:,:,:3]) #Create a mask for remaining 'void' colors.
    #Make another mask for remaining red that is < 9 and not uniform(there's got to be a better way)
    mask1 = (data[:,:,0] != data[:,:,1]) | (data[:,:,0] != data[:,:,2]) | (data[:,:,1] != data[:,:,2])
    mask[:,:,0] = mask1[:,:]
    mask[:,:,1] = mask1[:,:]
    mask[:,:,2] = mask1[:,:]
    data[:,:,:3][mask] = 0 #Set 'void' colors to black.

    return data

## Create mask for each screenshot

In [258]:
path = Path('./data/imageSeg/labels/screenshots')
savePath = Path('./data/imageSeg/labels')

# for each file in the lables/screenshots directory, resize, convert to an array, make the coded mask, then save to the
# labels directory.
for filename in os.listdir(path):
    if not (filename.endswith('.png')):
        continue
    
    #Open and resize image
    img = Image.open(path/filename).resize((760, 361)).convert(mode='RGB')
    
    #Make the mask, convert to PIL image, add _P to filename, and save in 'labels' folder
    label = makeMaskFast(img)
    img = Image.fromarray(label, mode='RGB')
    filenameP = filename[:-4] + '_P.png'
    img.save(savePath/filenameP, 'png')

# C.1: Resize screengrabs from Unreal

The screengrabs from Unreal are too big for the CNN I'm using. Size them down so they don't take up so much memory

In [80]:
path = Path('./data/imageSeg/images')

for filename in os.listdir(path):
    if not (filename.endswith('.png')):
        continue
        
    img = Image.open(path/filename).resize((760, 361))
    img.save(path/filename, 'png')

## D.1: Create validation set list and print to txt file

Split by name for validation set. This prints all the files names in the 'images' directory to the text file 'valid.txt'. After running this code edit valid.txt to include only the files you'd like in your validation set.

In [276]:
path = Path('./data/imageSeg/images')

with open("valid.txt", "w") as text_file:
    for filename in os.listdir(path):
        print(filename, file=text_file)

## D. 2: Create validation set list for image regression and print to txt file

Same as above, but grabs the file names from the 'labels' directory and writes it to 'valid_mask.txt'. Edit this text file to include only the files you'd like to include in your validation set.

In [4]:
path = Path('./data/imageSeg/labels')

with open("valid_mask.txt", "w") as text_file:
    for filename in os.listdir(path):
        print(filename, file=text_file)